<a href="https://colab.research.google.com/github/ImagingDataCommons/IDC-Tutorials/blob/master/notebooks/analysis/MedSAM_with_IDC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Experiment with Segmenting Anything in IDC using MedSAM

The goal of this notebook is to demonstrate how you can use NCI Imaging Data Commons as a source of real cancer imaging data to experiment with the MedSAM package.

**[NCI Imaging Data Commons (IDC)](https://imaging.datacommons.cancer.gov)** is a cloud-based environment containing publicly available cancer imaging data co-located with the analysis and exploration tools and resources. IDC is a node within the broader [NCI Cancer Research Data Commons (CRDC)](https://datacommons.cancer.gov/) infrastructure that provides secure access to a large, comprehensive, and expanding collection of cancer research data.

**[MedSAM](https://github.com/bowang-lab/MedSAM)** is a foundation model for promptable medical image segmentation introduced in the following manuscript:

> Ma, J., He, Y., Li, F., Han, L., You, C. & Wang, B. _Segment anything in medical images_. Nature Communications 15, 1–9 (2024). http://dx.doi.org/10.1038/s41467-024-44824-z

Please refer to the maniscript and repository above to learn more about MedSAM.

This notebook is based on the notebook developed by the MedSAM authors and originally shared here: https://github.com/bowang-lab/MedSAM/blob/main/tutorial_quickstart.ipynb.

Compared to the original notebook, we add the functionality to select radiology and microscopy images from IDC to better understand performance of this model.

Note that this notebook is intended to serve simply as a playground to experiment with MedSAM applied to a variety of images of different modalities and organs. It is not intended to facilitate saving of the resulting segmentations, or processing large volumes of data using MedSAM!

If you have any questions about this tutorial, please post your questions on the [IDC user forum](https://discourse.canceridc.dev/) (preferred) or email IDC support at support@canceridc.dev!

You can find more IDC notebooks and tutorials here: https://github.com/ImagingDataCommons/IDC-Tutorials/.

--

Authored by Andrey Fedorov and Daniela Schacherer

Initial version: February 2024

## Prerequisites

**If you are running this notebook on Google Colab, make sure you chose the GPU runtime - this notebook will not work on a CPU runtime!**

In the following cell we install the prerequisitie packages. This will take a few minutes.


In [ ]:
%%capture
!pip install git+https://github.com/bowang-lab/MedSAM.git

!pip install idc-index --upgrade
!pip install ipywidgets
!pip install itk
!pip install wsidicom
!pip install pydicom

## Setup MedSAM widget

First we download the pre-trained model shared by the MedSAM authors.

In [ ]:
# download model and data
!wget https://github.com/ImagingDataCommons/IDC-Tutorials/releases/download/0.2.0/medsam_vit_b.pth
!wget https://raw.githubusercontent.com/bowang-lab/MedSAM/main/utils/demo.py

Next we initiallize the model and setup the widget that will be later used for interacting with the loaded image.

In [ ]:
%matplotlib widget
from segment_anything import sam_model_registry
from demo import BboxPromptDemo
from google.colab import output
output.enable_custom_widget_manager()
MedSAM_CKPT_PATH = "medsam_vit_b.pth"
device = "cuda:0"
medsam_model = sam_model_registry['vit_b'](checkpoint=MedSAM_CKPT_PATH)
medsam_model = medsam_model.to(device)
medsam_model.eval()

## Experimenting with radiology images

We will use the [`idc-index`](https://github.com/ImagingDataCommons/idc-index) python package for selecting and downloading images from IDC. This package was installed as a prerequisite earlier.

To use `idc-index`, we first need to instantiate the client.



In [ ]:
from idc_index import index

idc_client = index.IDCClient()

Next we will select a subset of radiology DICOM modalities that correspond to images (this way we do not attempt to load non-image modalties, such as SR (Structured Reports) or RTSTRUCT (Radiotherapy Structure Sets)), and display the values of `BodyPartExamined` DICOM tag in the selector, which corresponds in general to the body part imaged.

In [ ]:
import ipywidgets as widgets

modalities_considered = ['CT','MR','PT','XA','DX']

body_parts = idc_client.index[idc_client.index['Modality'].isin(modalities_considered)]['BodyPartExamined'].unique()
body_part_picker = widgets.Dropdown(options=body_parts, value=body_parts[1])

display(body_part_picker)

The following cell will pick a random DICOM series from the list of the imaging modalities (CT, MR, PT, XA, DX, US) that has `BodyPartExamined` assigned the value selected above.

Note:
* if you want to pick a different series, just rerun the following cell
* if you would like to pick images of a specific modality, just modify the `modalities_considered` variable in the cell below, and rerun that cell

In [ ]:
import random, os
import pandas as pd

matching_series_list = idc_client.index[idc_client.index['Modality'].isin(modalities_considered) \
                                        & (idc_client.index['BodyPartExamined'] == body_part_picker.value)]['SeriesInstanceUID'].values

# select random series from the list
random_series_uid = random.choice(matching_series_list)
random_series_viewer_url = idc_client.get_viewer_URL(random_series_uid)

from IPython.display import IFrame
IFrame(random_series_viewer_url, width=1600, height=900)

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import nibabel as nib
import itk

# select random slice from the series above
random_image_slice_url = random.choice(idc_client.get_series_file_URLs(random_series_uid))

# download that random image slice
s5cmd = idc_client.s5cmdPath
samples_dir = 'idc_samples'
!mkdir -p $samples_dir
! $s5cmd --no-sign-request --endpoint-url https://s3.amazonaws.com cp --show-progress $random_image_slice_url $samples_dir

random_image_slice_file = os.path.join(os.getcwd(), samples_dir, random_image_slice_url.split('/')[-1])


random_image_slice_file_png = random_image_slice_file + ".png"

if os.path.exists(random_image_slice_file):
  img = itk.imread(random_image_slice_file)
  img_array = itk.array_view_from_image(img)
  #img_array = np.moveaxis(img, 0, -1)
  img_array_2d = img_array[0, :, :]

  # Define the output minimum and maximum intensity values
  output_min = 0
  output_max = 255

  # Rescale the array values to the range of itkImageUC2
  rescaled_array = (img_array_2d - img_array_2d.min()) / (img_array_2d.max() - img_array_2d.min()) * (output_max - output_min) + output_min

  # Convert the rescaled array to unsigned 8-bit integer type
  rescaled_array = rescaled_array.astype(np.uint8)

  rescaled_image = itk.GetImageFromArray(rescaled_array)

  # Define the output pixel type
  OutputPixelType = itk.UC
  OutputImageType = itk.Image[OutputPixelType, 2]


  itk.imwrite(rescaled_image, random_image_slice_file_png)

else:
  print('Failed to get the sample from IDC!')

bbox_prompt_demo = BboxPromptDemo(medsam_model)
bbox_prompt_demo.show(random_image_slice_file_png)

## Experimenting with slide microscopy images

In the following cell we initialize the list with the collections that contain DICOM SM (Slide Microscopy) modality.

There is more metadata to identify slides corresponding to specific tissue types, but it is currently not available via `idc-index`, which provides access to a small subset of metadata. You can check out the BigQuery ["getting started" tutorials](https://github.com/ImagingDataCommons/IDC-Tutorials/blob/master/notebooks/getting_started) to learn how to access full metadata index, but it is not essential for this notebook.



In [ ]:
from idc_index import index
import ipywidgets as widgets

idc_client = index.IDCClient()

collections = idc_client.index[idc_client.index['Modality']=='SM']['collection_id'].unique()
collection_picker = widgets.Dropdown(options=collections)

display(collection_picker)

In the next cell we will randomly pick a slide from the collection you selected above, and will generate a link that you can click to open that series directly in the notebook cell.

**You can rerun the following cell as much as you like to pick a different slide!**

In [ ]:
import random, os
import pandas as pd

matching_series_list = idc_client.index[(idc_client.index['collection_id'] == collection_picker.value) & (idc_client.index['Modality'] == 'SM')]['SeriesInstanceUID'].values

# select random series from the list
random_series_uid = random.choice(matching_series_list)
random_series_viewer_url = idc_client.get_viewer_URL(random_series_uid)

from IPython.display import IFrame
IFrame(random_series_viewer_url, width=1600, height=900)

Next we will download the hi-res layer of the selected slide image. We will then read the image using the open source [`wsidicom`](https://pypi.org/project/wsidicom/) python package.

In [ ]:
print("Getting largest instance (resolution layer) for series "+random_series_uid)

from matplotlib import pyplot as plt
import numpy as np
import subprocess as sp
#import nibabel as nib
#import itk

# select random slice from the series above
series_file_urlss = idc_client.get_series_file_URLs(random_series_uid)

# given bucket folder, get image sizes for all files
largest_instance = None
largest_instance_size = 0
for f in series_file_urlss:
  process = sp.run([idc_client.s5cmdPath,'--no-sign-request','--endpoint-url','https://s3.amazonaws.com','du',f],capture_output=True)
  instance_size = int(process.stdout.decode('utf-8').split(' ')[0])
  if instance_size > largest_instance_size:
    largest_instance = f
    largest_instance_size = instance_size

print("Largest instance size: "+str(largest_instance_size))
print("Largest instance URL: "+largest_instance)

# download that random image slice
s5cmd = idc_client.s5cmdPath
samples_dir = 'idc_samples'
!mkdir -p $samples_dir
! $s5cmd --no-sign-request --endpoint-url https://s3.amazonaws.com cp --show-progress $largest_instance $samples_dir

random_image_slice_file = os.path.join(os.getcwd(), samples_dir, largest_instance.split('/')[-1])

print('Downloaded {}'.format(largest_instance))

# load the downloaded image using wsidicom
from wsidicom import WsiDicom
slide = WsiDicom.open(random_image_slice_file)


Now that the image is loaded in memory, we will randomly sample a rectangular region (tile) of size 200x200 pixels to use with the MedSAM demonstration.


Note:
* You can rerun this cell as many times as you like to sample a different region
* If you see a blank tile, most likely this is because the random sample happened to be outside of the tissue area - just rerun the cell until you see a non-empty tile

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Display the image
tile_size = 200

corner_x = random.randint(0, slide.size.width-tile_size)
corner_y = random.randint(0, slide.size.height-tile_size)

region = slide.read_region(size=(tile_size, tile_size), level=0, location=(corner_x, corner_y))

region.save('tile.png')

bbox_prompt_demo = BboxPromptDemo(medsam_model)
bbox_prompt_demo.show('tile.png')

## More information about MedSAM

These pointers are carried over from the [original MedSAM tutorial notebook](https://github.com/bowang-lab/MedSAM/blob/main/tutorial_quickstart.ipynb):
   - How was MedSAM trained? or How to fine-tune SAM on customized datasets? [training](https://github.com/bowang-lab/MedSAM/tree/main#model-training)
   - How to fine-tune the model with text-based prompts: [training](https://github.com/bowang-lab/MedSAM/tree/main/extensions/text_prompt) and [inference colab](https://colab.research.google.com/drive/1wexPLewVMI-9EMiplfyoEtGGayYDH3tt?usp=sharing)
   - How to fine-tune the model with point-based prompts: [training](https://github.com/bowang-lab/MedSAM/tree/main/extensions/point_prompt) and [inference colab](https://colab.research.google.com/drive/1cCBw_IhdPiWE4sN7QwqKJPgAFlWsKgkm?usp=sharing)


## What's next?

You can find more IDC notebooks and tutorials here: https://github.com/ImagingDataCommons/IDC-Tutorials/tree/master.

You can contact IDC support by sending email to support@canceridc.dev or posting your question on [IDC User forum](https://discourse.canceridc.dev).


## Acknowledgments

We gratefully acknowledge the authors of the [MedSAM](https://github.com/bowang-lab/MedSAM) package for developing the open source implementation of their method and sharing the demonstration notebook!

Imaging Data Commons has been funded in whole or in part with Federal funds from the National Cancer Institute, National Institutes of Health, under Task Order No. HHSN26110071 under Contract No. HHSN261201500003l.

If you use IDC in your research, please cite the following publication:

> Fedorov, A., Longabaugh, W. J. R., Pot, D., Clunie, D. A., Pieper, S. D., Gibbs, D. L., Bridge, C., Herrmann, M. D., Homeyer, A., Lewis, R., Aerts, H. J. W. L., Krishnaswamy, D., Thiriveedhi, V. K., Ciausu, C., Schacherer, D. P., Bontempi, D., Pihl, T., Wagner, U., Farahani, K., Kim, E. & Kikinis, R. _National cancer institute imaging data commons: Toward transparency, reproducibility, and scalability in imaging artificial intelligence_. Radiographics 43, (2023). http://dx.doi.org/10.1148/rg.230180